# Import Dataset

In [68]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from Preprocessing_functions import *



In [69]:
train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')
test_data = pd.read_csv('test_data.csv', index_col='Claim Identifier')

/var/folders/z6/_vn8cndd4x5dmx2cpjfdkjdr0000gn/T/ipykernel_30731/3470921380.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')


In [70]:
train_data.isna().sum()

Accident Date                          23134
Age at Injury                          19445
Alternative Dispute Resolution         19445
Assembly Date                              0
Attorney/Representative                19445
Average Weekly Wage                    48096
Birth Year                             48523
C-2 Date                               34005
C-3 Date                              406226
Carrier Name                           19445
Carrier Type                           19445
Claim Injury Type                      19445
County of Injury                       19445
COVID-19 Indicator                     19445
District Name                          19445
First Hearing Date                    442673
Gender                                 19445
IME-4 Count                           460668
Industry Code                          29403
Industry Code Description              29403
Medical Fee Region                     19445
OIICS Nature of Injury Description    593471
WCIO Cause

# Preprocessing

In [71]:
train_data = train_data[~(train_data.drop(columns=['Assembly Date']).isna().all(axis=1) & train_data['Assembly Date'].notna())]

In [72]:
X = train_data.drop(columns=['Claim Injury Type', 'WCB Decision', 'Agreement Reached'])
y = train_data['Claim Injury Type']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [73]:
y.isna().sum()

np.int64(0)

In [74]:
CODE_COLUMNS = ['Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

DESCRIPTION_COLUMNS = ['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description']

BOOLEAN_COLUMNS = ['Alternative Dispute Resolution', 'Attorney/Representative','COVID-19 Indicator']

date_order = ['Accident Date', 'C-2 Date','C-3 Date','Assembly Date', 'First Hearing Date']


In [75]:
numerical_columns = [
    'Accident Date', 
    'Age at Injury', 
    'Assembly Date', 
    'Average Weekly Wage', 
    'Birth Year', 
    'C-2 Date', 
    'C-3 Date', 
    'First Hearing Date', 
    'IME-4 Count', 
]

categorical_features = ['Alternative Dispute Resolution',
 'Attorney/Representative',
 'Carrier Name',
 'Carrier Type',
 'County of Injury',
 'COVID-19 Indicator',
 'District Name',
 'Gender',
 'Industry Code',
 'Medical Fee Region',
 'WCIO Cause of Injury Code',
 'WCIO Nature of Injury Code',
 'WCIO Part Of Body Code',
 'Zip Code']

col_minmax = ['Age at Injury',
               'Birth Year', 
               'Number of Dependents']

col_standart = ['Accident Date',
                'Assembly Date',
                'Average Weekly Wage',
                ]

low_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() < 10]
high_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() > 10]




In [76]:
X_train[categorical_features] = X_train[categorical_features].astype(str)
X_val[categorical_features] = X_val[categorical_features].astype(str)

In [77]:
def drop_description_columns(X_train, X_val):
    """
    Drop all columns in X_train and X_val that contain the word 'description' in their names (case-insensitive).
    """
    description_columns = X_train.columns[X_train.columns.str.contains('description', case=False, na=False)]
    

    X_train = X_train.drop(description_columns, axis=1)
    X_val = X_val.drop(description_columns, axis=1)
    
    return X_train, X_val

X_train ,X_val = drop_description_columns(X_train, X_val)

In [ ]:
def preprocessing_dum(X_train, X_val):
    convert_to_timestamp(X_train, X_val, date_order)
    convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    impute_mean_numerical(X_train, X_val, numerical_columns)
    fill_missing_with_mode(X_train, X_val)
    feature_creation_has_Cdate(X_train, X_val)
    X_train = X_train.drop(columns=['C-3 Date', 'First Hearing Date', 'C-2 Date', 'IME-4 Count'])
    X_val = X_val.drop(columns=['C-3 Date', 'First Hearing Date', 'C-2 Date', 'IME-4 Count'])

    return X_train, X_val



def scaling_encoding(X_train, X_val):
    scaling_minmax(X_train, X_val, col_minmax)
    scaling_standard(X_train, X_val, col_standart)
    X_train, X_val = encoding_onehot(X_train, X_val, low_cardinality_cols)
    encoding_frequency1(X_train, X_val, high_cardinality_cols)

    return X_train, X_val

X_train, X_val = preprocessing_dum(X_train, X_val)

X_train.info()


/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:320: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  def impute_mode_categorical(X_train, X_val, columns):
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:321: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will nev

KeyError: 'GREAT NORTHERN INSURANCE CO.'

In [34]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459220 entries, 5785935 to 6027959
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Accident Date                   459220 non-null  float64
 1   Age at Injury                   459220 non-null  float64
 2   Alternative Dispute Resolution  459216 non-null  object 
 3   Assembly Date                   459220 non-null  float64
 4   Attorney/Representative         459220 non-null  object 
 5   Average Weekly Wage             459220 non-null  float64
 6   Birth Year                      459220 non-null  float64
 7   C-2 Date                        459220 non-null  float64
 8   C-3 Date                        459220 non-null  float64
 9   Carrier Name                    459220 non-null  object 
 10  Carrier Type                    459220 non-null  object 
 11  County of Injury                459220 non-null  object 
 12  COVID-19 Indic

In [59]:

"""
def preprocessing_complex(X_train, X_val):
    convert_to_timestamp(X_train, X_val, date_order)
    convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    impute_mean_numerical(X_train, X_val, ['C-2 Date'])
    fillna_zip_code(X_train, X_val)
    fillnan_accident_date(X_train,X_val)
    fillnan_birth_year(X_train,X_val)
    feature_creation_has_Cdate (X_train, X_val)
    fill_missing_with_mode(X_train, X_val)
    drop_description_columns(X_train, X_val)
    scaling_standard(X_train, X_val, columns)
    encoding_onehot(X_train, X_val, columns)
    return X_train, X_val
preprocessing_complex(X_train, X_val)

X_train.info()
"""





"\ndef preprocessing_complex(X_train, X_val):\n    convert_to_timestamp(X_train, X_val, date_order)\n    convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)\n    impute_mean_numerical(X_train, X_val, ['C-2 Date'])\n    fillna_zip_code(X_train, X_val)\n    fillnan_accident_date(X_train,X_val)\n    fillnan_birth_year(X_train,X_val)\n    feature_creation_has_Cdate (X_train, X_val)\n    fill_missing_with_mode(X_train, X_val)\n    drop_description_columns(X_train, X_val)\n    scaling_standard(X_train, X_val, columns)\n    encoding_onehot(X_train, X_val, columns)\n    return X_train, X_val\npreprocessing_complex(X_train, X_val)\n\nX_train.info()\n"

# Model Training

In [60]:
X_val.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5947783,2022-03-14,51.0,N,2022-03-22,N,0.0,1970.0,2022-03-22,2022-05-20,CANTON CENTRAL SCHOOL DISTRICT,...,NaN,F,NaN,61.0,I,25.0,10.0,-9.0,13652,2.0
6150876,2022-12-02,61.0,N,2022-12-09,Y,NaN,1961.0,2022-12-09,NaN,STATE INSURANCE FUND,...,2024-04-02,M,1.0,48.0,I,56.0,52.0,38.0,14416,2.0
5397365,2011-06-14,50.0,N,2020-01-08,Y,0.0,1961.0,2020-01-07,NaN,COMMERCE AND INDUSTRY INS CO,...,NaN,F,NaN,62.0,IV,25.0,49.0,55.0,11726,1.0
6077399,2022-02-07,54.0,N,2022-09-07,Y,1502.5,1967.0,2022-09-13,2022-08-31,STATE INSURANCE FUND,...,2023-11-09,M,5.0,62.0,IV,32.0,10.0,-9.0,11565,0.0
5945251,2022-03-15,35.0,N,2022-03-16,N,0.0,1986.0,2022-03-16,NaN,SAFETY NATIONAL CASUALTY CORP,...,NaN,U,NaN,62.0,IV,60.0,52.0,41.0,11212,0.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE

# Reset indices if needed
X = X.reset_index(drop=True)  # Ensure sequential indexing
y = y.reset_index(drop=True)

# Define range of k
k_range = range(1, 10)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

mean_accuracies = []

for k in k_range:
    accuracies = []
    for train_index, test_index in kf.split(X):
        # Use .iloc to slice data based on train/test splits
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Apply preprocessing
        X_train, X_test = preprocessing_dum(X_train, X_test)
        X_train, X_test = scaling_encoding(X_train, X_test)
        
        # Feature selection
        X_train_selected, selected_features, feature_ranking = feature_selection_rfe(X_train, y_train, 10 , LogisticRegression())
        
        # Train KNN model
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train_selected, y_train)
        
        # Evaluate on the test set
        y_pred = knn.predict(X_test_selected)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    # Compute mean accuracy for this k
    mean_accuracies.append(np.mean(accuracies))

# Find the optimal k
optimal_k = k_range[np.argmax(mean_accuracies)]
print(f"The optimal number of neighbors is {optimal_k}.")

# Plot the results
plt.plot(k_range, mean_accuracies)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Mean Accuracy')
plt.title('Optimal k Selection using K-Fold Cross-Validation')
plt.show()

# Final training with optimal k
X_preprocessed, _ = preprocessing_dum(X, X)  # Apply preprocessing to the full dataset
X_scaled, _ = scaling_encoding(X_preprocessed, X_preprocessed)
selector = RFE(estimator=LogisticRegression(), n_features_to_select=10)
X_final = selector.fit_transform(X_scaled, y)
final_knn = KNeighborsClassifier(n_neighbors=optimal_k)
final_knn.fit(X_final, y)

print(f"Model trained with optimal k={optimal_k}.")


  

/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = pd.to_datetime(X_train[col], errors='coerce')
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[col] = pd.to_datetime(X_val[col], errors='coerce')
/Users/philippedutranoit/document ordi/nova/machine learning /project/

ValueError: could not convert string to float: 'NEW HAMPSHIRE INSURANCE CO'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Range of k values to test (for XGBoost, we can try different hyperparameters)
learning_rate_range = [0.01, 0.1, 0.3, 0.5, 0.7]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store mean accuracy for each learning rate
mean_accuracies = []

# Perform K-Fold for each learning rate
for learning_rate in learning_rate_range:
    accuracies = []
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets for this fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the XGBoost model
        model = XGBClassifier(learning_rate=learning_rate, use_label_encoder=False, eval_metric='mlogloss')
        model.fit(X_train, y_train)
        
        # Evaluate on the test set
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    # Compute the mean accuracy for this learning rate
    mean_accuracies.append(np.mean(accuracies))

# Find the optimal learning rate
optimal_lr = learning_rate_range[np.argmax(mean_accuracies)]
print(f"The optimal learning rate is {optimal_lr}.")

# Plot the results
plt.plot(learning_rate_range, mean_accuracies)
plt.xlabel('Learning Rate')
plt.ylabel('Mean Accuracy')
plt.title('Optimal Learning Rate Selection using K-Fold Cross-Validation (XGBoost)')
plt.show()

# Final training with optimal learning rate
final_model = XGBClassifier(learning_rate=optimal_lr, use_label_encoder=False, eval_metric='mlogloss')
final_model.fit(X, y)
print(f"Model trained with optimal learning rate={optimal_lr}.")
